In [1]:
import os
import shutil
import pandas as pd

csv_file = 'metadata.csv'
df = pd.read_csv(csv_file)

image_folder = 'testing'
benign_folder = 'benign_testing'
malignant_folder = 'malignant_testing'

os.makedirs(benign_folder, exist_ok=True)
os.makedirs(malignant_folder, exist_ok=True)

for index, row in df.iterrows():
    isic_id = row['isic_id']
    benign_malignant = row['benign_malignant']
    
    image_name = f"{isic_id}.jpg"
    source_path = os.path.join(image_folder, image_name)
    
    if os.path.exists(source_path):
        if benign_malignant == 'benign':
            shutil.copy(source_path, benign_folder)
        elif benign_malignant == 'malignant':
            shutil.copy(source_path, malignant_folder)
    else:
        print(f"Image {image_name} not found.")



ModuleNotFoundError: No module named 'pandas'

In [ ]:
import os
import shutil
import random

def split_data(source_benign, source_malignant, target_folder, train_ratio=0.8):
    os.makedirs(os.path.join(target_folder, 'benign_training'), exist_ok=True)
    os.makedirs(os.path.join(target_folder, 'benign_testing'), exist_ok=True)
    os.makedirs(os.path.join(target_folder, 'malignant_training'), exist_ok=True)
    os.makedirs(os.path.join(target_folder, 'malignant_testing'), exist_ok=True)

    benign_images = [f for f in os.listdir(source_benign) if f.endswith(('.jpg', '.png'))]
    malignant_images = [f for f in os.listdir(source_malignant) if f.endswith(('.jpg', '.png'))]

    random.shuffle(benign_images)
    random.shuffle(malignant_images)

    num_benign = len(benign_images)
    num_benign_train = int(num_benign * train_ratio)
    benign_train = benign_images[:num_benign_train]
    benign_test = benign_images[num_benign_train:]

    num_malignant = len(malignant_images)
    num_malignant_train = int(num_malignant * train_ratio)
    malignant_train = malignant_images[:num_malignant_train]
    malignant_test = malignant_images[num_malignant_train:]

    for image in benign_train:
        shutil.copy(os.path.join(source_benign, image), os.path.join(target_folder, 'benign_training', image))

    for image in benign_test:
        shutil.copy(os.path.join(source_benign, image), os.path.join(target_folder, 'benign_testing', image))

    for image in malignant_train:
        shutil.copy(os.path.join(source_malignant, image), os.path.join(target_folder, 'malignant_training', image))

    for image in malignant_test:
        shutil.copy(os.path.join(source_malignant, image), os.path.join(target_folder, 'malignant_testing', image))

    print(f"Data split completed: ")
    print(f"Benign Training: {len(benign_train)} images")
    print(f"Benign Testing: {len(benign_test)} images")
    print(f"Malignant Training: {len(malignant_train)} images")
    print(f"Malignant Testing: {len(malignant_test)} images")

source_benign_folder = "testing/benign"
source_malignant_folder = "testing/malignant"
target_folder = "image_split"

split_data(source_benign_folder, source_malignant_folder, target_folder)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

img_height, img_width = 224, 224
batch_size = 8
epochs = 10
num_classes = 2

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    'image_split/training',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'image_split/training',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

base_model.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)
# save model as keras
model.save('model.keras', save_format='keras')



In [ ]:


model = tf.keras.models.load_model('model.keras') 


converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

